In [ ]:
%matplotlib
import numpy as np
import pandas as pd
import os
import sys

master list includes all participants collected, intervention type, problem list at scan

In [ ]:
master_list_path='/Users/daelsaid/scratch/scsnl_data_scratch/mri_project_update/'
master=os.path.join(master_list_path,'master_intervention_grouping_list.csv')
sym=os.path.join(master_list_path,'sym_mri.csv')
grid=os.path.join(master_list_path,'grid_mri.csv')

define dataframes for various CSVS

In [ ]:
grid_df=pd.read_csv(grid,dtype='str')
sym_df=pd.read_csv(sym,dtype='str')
master_df=pd.read_csv(master,dtype='str').drop_duplicates()

sym_multisess_subjectlist_df=pd.read_csv(os.path.join(master_list_path,'sym_multisession_subjectlist.csv'),dtype=str)
grid_multisess_subjectlist_df=pd.read_csv(os.path.join(master_list_path,'grid_multisession_subjectlist.csv'),dtype=str)

new_df=pd.DataFrame()

merge CSVS into 1 master df that includes the following columns:

'PID', 'visit', 'training_group', 'problem_order', 'intervention', 'Visit_sym', 'Session_sym', 'Run1_sym', 'Run2_sym', 'Run3_sym', 'Run4_sym', 'Contrast_File_sym', 'good_runs_sym', 'Visit_grid', 'Session_grid', 'Run1_grid', 'Run2_grid', 'Run3_grid', 'Run4_grid', 'Contrast_File_grid', 'good_runs_grid'

In [ ]:
intermediate_merged=pd.merge(master_df,sym_multisess_subjectlist_df,left_on=['PID','visit'],right_on=['PID','Visit'],how='left',copy=False)
final_merged_master_df=pd.merge(intermediate_merged.reset_index(),grid_multisess_subjectlist_df,left_on=['PID','visit'],right_on=['PID','Visit'],how='left',copy=False,suffixes=['_sym','_grid'])

### Average "good scans" passing QC cutoff grouped by usability, visit, and task\

Criteria for usable runs:

x,y,z pitch, roll, yaw <10  
Max Displacement<10  
MeanScan-to-Scan Displacement (FD) <0.5  
Volume repair <15  
Good behavioral accuracy >33%  
Good functional image quality on unnormalized image (meets okay or good criteria)

In [ ]:
movementstats_df=pd.read_csv(os.path.join(master_list_path,'movement_stats_subject_runlist.csv'))
used_runs_grp1=pd.read_csv(os.path.join(master_list_path,'pre_vs_post_group_subjectlist.csv'),dtype=str)
used_runs_grp2=pd.read_csv(os.path.join(master_list_path,'group_subjlist_visitxintervention.csv'),dtype=str)
used_runs=pd.merge(used_runs_grp1,used_runs_grp2, on=['PID','visit','session'],how='left')
movementstats_df.drop_duplicates(inplace=True)


behav_data=pd.read_csv(os.path.join(master_list_path,'raw_behav/data_summary.csv'))
movementstats_behav_df=pd.DataFrame()
movementstats_behav_df=movementstats_df.merge(behav_data,left_on=['PID','visit','session','task'],right_on=['scanid','visit','session','taskname'])


new_df=movementstats_behav_df.loc[:,'usability':'rtcorrect.mean']


In [ ]:
for idx,subj in movementstats_df.groupby(['visit','task','usability']):
    print idx[2],idx[0],idx[1],subj.PID.tolist()
    df1=pd.merge(final_merged_master_df,subj.astype(str).reset_index(), on=['PID','task'],how='left',copy=False)
    df2=pd.merge(df1.reset_index(),used_runs,on=['PID','visit','session'],how='left',copy=False)

Intervention by visit x training group subject count breakdown

In [ ]:
for idx,grouping in final_merged_master_df.groupby(['visit']):
    print 'VISIT',idx[0]
    print grouping['PID'].drop_duplicates().tolist()

In [ ]:
for idx,grouping in final_merged_master_df.groupby(['visit','intervention']):
    if all(grouping.visit=='1'):
        print idx[1]+' PRE'
        print grouping.PID.drop_duplicates().tolist()
        print ''
        print grouping.good_runs_grid.dropna().values
    if all(grouping.visit=='2'):
        print ''
        print idx[1]+' POST'
        print grouping.PID.drop_duplicates().tolist()

In [ ]:
# # final_merged_master_df.drop_duplicates().groupby(['intervention','visit','training_group']).describe()
for x,y in final_merged_master_df.groupby(['visit','intervention']):
    print y.PID.to_csv("test.csv")

In [ ]:

for x,y in movementstats_behav_df.groupby(['PID','usability','visit','taskname']):
    if x[1] == 0 and x[2] == 1:
        print x[0],x[2],x[3],y[['accuracy.mean','rtcorrect.mean']].mean()
    

In [ ]:
movementstats_behav_df.groupby(['usability','visit','taskname'])[['accuracy.mean','rtcorrect.mean']].mean()

In [ ]:
for idx,task in behav_data.groupby(['visit','taskname']):
    print idx[0], idx[1],task['accuracy.mean'].mean().plot()

In [ ]:
fmri_list='/Users/daelsaid/Desktop/fmri_list.csv'
movementstats='/Users/daelsaid/Desktop/MovementSummaryStats.csv'

movedf=pd.read_csv(movementstats,dtype=str)
fmrilist_df=pd.read_csv(fmri_list,dtype='str')

subjectlist=[]
subj_visit_session_task=[]

print fmrilist_df

In [ ]:
for subj,val in fmrilist_df.groupby(['PID','visit','session']):
    subj_visit_session_task.append([subj[0],subj[1],subj[2],val['task'].T.tolist()])

for subj in subj_visit_session_task:
    print sorted(subj[0:len(subj)])

for pid,stats in movedf.groupby(['PID','Visit']):
    subjectlist.append([pid,stats['TASK'].tolist()])

print subjectlist

for subj,run in movedf.groupby(['PID','Visit']):
    print subj[0],run['TASK'].tolist()
for idx,s in enumerate(subjectlist):
    print idx
    subj=s[0][0]
    visit=s[0][1]
    sym_list=[]
    grid_list=[]
    print subj,sorted(s[1])
